# Preprocessing and Feature Engineering for Objects Dataset

- Clean Objects DataFrame
- Create new , significant features 
- OneHotEncode 

In [ ]:
# start by removing redundant columns 
list(objects2.columns)

In [ ]:
drop = ['twitter_username',
 'logo_url', ]

objects2 = objects2.drop(drop, axis=1)

In [ ]:
# we only need one column for the location , find the best one to OneHotEncode
loc_cols = [ 'country_code','state_code','city','region']
objects2[loc_cols]

In [ ]:
# region is definitely too specific for OHC , but let's explore distinct value count in other columns
objects2['country_code'].value_counts().sum()

In [ ]:
objects2['state_code'].value_counts().sum()

In [ ]:
objects2['city'].value_counts().sum()

- state_code has the least distinct values because it only rpovides a value for countries with states
-  best option is so aggregate country_code into high level continents

In [ ]:
objects2['country_code'].value_counts()

These codes are part of the ISO 3166-1 standard and are commonly used to uniquely identify countries and territories 
Credits to https://statisticstimes.com/geography/countries-by-continents.php for the excel file for mapping 

In [ ]:
df = pd.read_excel(r"C:\Users\Admin\Final-Project\ISO Continents.xlsx")

In [ ]:
map=['ISO-alpha3 Code','Continent']
mapping =df[map]

In [ ]:
objects3 = objects2.copy()

In [ ]:
continent_list = []

for code in objects3['country_code']:
    if code in df['ISO-alpha3 Code'].values:
        region = df.loc[df['ISO-alpha3 Code'] == code, 'Continent'].values[0]
        continent_list.append(region)
    else:
        continent_list.append(None)

objects3['continent'] = continent_list

In [ ]:
objects3['continent'].value_counts()

In [ ]:
objects3.columns

In [ ]:
objects3.to_csv('objects3.csv')

In [ ]:
objects4= objects3.copy()

In [ ]:
len(list(objects4['category_code'].value_counts()))

In [ ]:
# create df with only location info , then isolate for NaN continent rows so can see if can impute . if no , drop these columns or categorize as 'virtual'
locationinfo = ['country_code', 'state_code', 'city', 'region','continent']
locationsdf = objects3[locationinfo]
locationsdf

In [ ]:
# exploring NaN continent rows
no_continent = locationsdf[locationsdf['continent'].isna()]
no_continent['city'].value_counts()

In [ ]:
no_continent['region'].value_counts()

In [ ]:
#explore rows with unknown regions
no_region = no_continent[no_continent['region']=='unknown']
no_region['city'].value_counts()

Its less than one percent of the data ,so i think I should skip the work for now because of time constraint.  
Now , let's assume NaN continents for the most part simply do not have a location / HQs . So , rewrite those businesses as 'Virtual'.

In [ ]:
objects5 = objects4.copy()

# Replace NaN values in the 'continent' column with 'Virtual'
objects5['continent'].fillna('Virtual', inplace=True)

# validate
objects5['continent'].value_counts()

In [ ]:
# validate
objects5['continent'].value_counts()

In [ ]:
intervals = ['updated_at', 'founded_at']
intervaldf = objects5[ intervals]
intervaldf

In [ ]:
sum(intervaldf['founded_at'].isnull())

In [ ]:
# created year is at shows a year (  if not the first year) in which the company was in existence so gives some idea of company's age although not 100% accurate
intervals3 = ['created_at', 'updated_at']
intervaldf3 = objects5[ intervals3]
intervaldf3

In [ ]:
# FIrst , convert both columns to DateTIme
objects5['created_at'] = pd.to_datetime(objects5['created_at'])
objects5['updated_at']= pd.to_datetime(objects5['updated_at'])

In [ ]:
# createing company tracking interval column
objects5['tracking_interval'] = objects5['updated_at'] - objects5['created_at']

In [ ]:
# validate 
objects5['tracking_interval']

In [ ]:
# find what kind of startups are under Person
person_type = objects[objects['entity_type']== 'Person']
person_type.head(25)

In [ ]:
remove all persons becasue theyre not startups

In [ ]:
objects6 = objects5.copy()

In [ ]:
objects6 = objects6[objects6['entity_type'] != 'Person']

In [ ]:
# find what kind of startups are under Peroduct
prod_type = objects[objects['entity_type']== 'Product']
prod_type.head(50)

In [ ]:
# check if products represent individual offerings of startups and therfore do not represent companies/startups themselves and therefore should be dropped
# if total funding is 0 , indicates that its not a a company
prod_type['funding_total_usd'].value_counts()

In [ ]:
# check further
prod_type.info()

category_code , description , short_description all completely null , while parent_id is almost completely full . Perhaps parent_Id represents the actual company behind the product.

In [ ]:
# check parent_id values
prod_type['parent_id']

Hypothesis is true . Product does not represent a type of startup , rather it represents the offering/product of the startups , and parent_id links it to its associated company.
Separate Products into a different dataframe to reference after MVP . 
For now , we will only work with the companies themselves.

In [ ]:
prod_type.to_csv('products.csv')
com_type.to_csv('companies6.csv')

In [ ]:
com_type = objects6[objects6['entity_type']== 'Company']
com_type

In [ ]:

sorted_com_type = com_type.sort_values(by='updated_at', ascending=False)

# Print the sorted DataFrame
sorted_com_type.head(3) # just want to see the most recent updates

In [ ]:
# Convert 'founded_at' column to datetime if it's not already
com_type['founded_at'] = pd.to_datetime(com_type['founded_at'], errors='coerce')

# Extract the year from 'founded_at' column
com_type['founded_year'] = com_type['founded_at'].dt.year

# Calculate the mean of the founded years
average_founded_year = com_type['founded_year'].mean()

# Print the calculated mean
print("Average Founded Year:", average_founded_year)

In [ ]:
sum(com_type['founded_year'].notna())

91227 out of 196553 have founding date values . 
conduct baseline model on all rows ie. removing founding dates , and then after only with those 90K rows and see if startup age (hypothesis 11.5 = closed) changes prediction score

-- now create model df

In [ ]:
# these are the baseline features 
columns = [ 'funding_rounds',
 'funding_total_usd','continent','tracking_interval','milestones','relationships', 'status']
model1 = com_type[columns]

In [ ]:
# aggregate the status column into target classes 
model1['status'].value_counts()

Strategy for imbalanced target class 'operating' ( which makes empirical sense since the tracking range is not long enough to capture the average lifespan of a startup):
>  Options:
- aggregate acquired and ipo into one target class 'Exit' since that's the goal for VCs  ( adjust loss function accordingly ie. false negatives for thsi class more punished)
- Resampling:

Oversampling: Increase the number of instances in the minority class by duplicating or creating synthetic samples. Techniques like SMOTE (Synthetic Minority Over-sampling Technique) can be effective.
Undersampling: Reduce the number of instances in the majority class by randomly selecting a subset of samples. This can help balance the class distribution.
Weighted Loss Function:

- Adjust the loss function in your classification algorithm to give more weight to the minority class. This can make the model more sensitive to the minority class during training.
Ensemble Methods:

- Use ensemble techniques like Random Forest, which can handle imbalanced classes better than individual models. Random Forest's construction of multiple decision trees can mitigate the class imbalance issue.
Anomaly Detection:

- Treat the minority class as an anomaly detection problem. This involves building a model to identify instances that deviate significantly from the majority class. Techniques like Isolation Forest and One-Class SVM can be useful.
Change the Evaluation Metric:

- Instead of using accuracy as the evaluation metric, consider metrics like precision, recall, F1-score, or area under the ROC curve (AUC-ROC) that provide a more comprehensive understanding of model performance for imbalanced classes.
- Model Selection:

Choose algorithms that inherently handle imbalanced data well, such as algorithms that allow you to set class weights or algorithms designed to work well with skewed distributions.


In [ ]:
# successful exit 
model2 = model1.copy()

# Create a new column 'successful_exit' based on 'status' conditions
model2['successful_exit'] = model2['status'].apply(lambda x: 'Exit' if x in ['acquired', 'ipo'] else x)

#validate
model2['successful_exit'].value_counts()

# Focused Exploratory Data Analysis: In-Depth Investigation and Distributions of Model Features
analysis goes beyond the initial EDA and delves deeper into specific aspects of the data, particularly focusing on features relevant to the  model

In [ ]:
model2 .describe()

 " 90% of startups fail" -  https://businessdatalist.com/how-long-does-a-startup-company-last

In [ ]:
# check if research holds true for model features

# True or False for out data?

import matplotlib.pyplot as plt

# Class distribution
class_counts = {
    'operating': 183441,
    'Exit': 10528,
    'closed': 2584
}

# Calculate the total number of instances
total_instances = sum(class_counts.values())

# Calculate the ratio of 'closed' class to all other classes
closed_ratio = class_counts['closed'] / total_instances

# Plotting
plt.bar(class_counts.keys(), class_counts.values())
plt.xlabel('Classes')
plt.ylabel('Number of Instances')
plt.title('Class Distribution')
plt.show()

# Print the ratio
print(f'Ratio of "closed" class to all other classes: {closed_ratio:.4f}')

False . Since this data is only a short tracking interval , only 1.3% of the startups failed. 
When researched some of these companies current status - many have shut down / bankrupt since bringing the percentage of failed companies closer to the  90% ratio , but still not at all there.
This could be because while 90% (roughly) of startups fail , only .05% receive VC funding . This select group may not be subject to the same failure rate.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only numeric features from your DataFrame
num = model3.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = num.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Create a heatmap using seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)

# Set the title of the heatmap
plt.title('Correlation Matrix Heatmap')

# Show the plot
plt.show()

makes  sense . the more tracking days , the more funding rounds and milestones the dataset will have captured

In [ ]:
import plotly.express as px

# Distribution of numeric features
fig_numeric = px.histogram(model3, x="funding_rounds", title="Distribution of Funding Rounds")

# Distribution of funding categories
fig_funding = px.histogram(model3, x="funding_total_bin", title="Distribution of Funding Categories")

# Show the interactive plots
fig_numeric.show()
fig_funding.show()

In [ ]:
fig_continents = px.histogram(model3, x="continent", title="Distribution of Continents")
fig_continents.show()


In [ ]:
fig_cities = px.histogram(objects, x="city", title="Distribution of Cities")
fig_cities.show()

In [ ]:
fig_category_codes = px.histogram(objects, x="category_code", title="Distribution of Category Codes")
fig_category_codes.show()

In [ ]:
fig_employee_count = px.histogram(model3, x="relationships", title="Distribution of Employee Counts")
fig_employee_count.show()


Create categorical bins for funding amount since many unknown values

In [ ]:
 Define custom function for assigning funding categories
def assign_funding_category(funding_total):
    if funding_total == 0:
        return 'Undisclosed Funding'
    elif 0 < funding_total <= low_funding:
        return 'Low Funding'
    elif low_funding < funding_total <= medium_funding:
        return 'Medium Funding'
    elif medium_funding < funding_total <= high_funding:
        return 'High Funding'

# Calculate mean and standard deviation of funding_total_usd
mean_funding = model2['funding_total_usd'].mean()
std_funding = model2['funding_total_usd'].std()

low_funding = mean_funding - std_funding
medium_funding = mean_funding
high_funding = model2['funding_total_usd'].max()

# Apply the custom function to create funding categories
model2['funding_total_bin'] = model2['funding_total_usd'].apply(assign_funding_category)

# validate
model2['funding_total_bin']

In [ ]:

# Convert 'tracking_interval' to total days
model2['tracking_interval_days'] = model2['tracking_interval'].dt.total_seconds() / (60 * 60 * 24)

In [ ]:
# Define features and target
X = model3.drop(columns=['successful_exit', 'Unnamed: 0'])
y = model3['successful_exit']